In [2]:
import os
import sys
sys.path.append(os.path.abspath('..')) # adds the parent directory of the current working directory to the list of places where Python will look for modules to import

In [12]:
# instead of restarting the kernel reload the modules
from importlib import reload
from scripts import data_load_clean_transform, feature_engineering, credit_scoring_rfms, credit_scoring_model
reload(data_load_clean_transform)
reload(feature_engineering)
reload(credit_scoring_rfms)
reload(credit_scoring_model)

<module 'scripts.credit_scoring_model' from 'c:\\ML and DS Files\\Kifiya AI\\Kaim-week-6\\scripts\\credit_scoring_model.py'>

In [15]:
from scripts.data_load_clean_transform import DataCleaner,DataLoader
from scripts.feature_engineering import FeatureEngineering
from scripts.credit_scoring_rfms import CreditScoringRFMS
from scripts.credit_scoring_model import CreditScoringModel 

In [5]:
file_path = ('../week 6 data/data/data.csv')
loader = DataLoader(file_path)
data = loader.load_csv()

2025-01-27 09:53:51,934 - INFO - Data successfully loaded from ../week 6 data/data/data.csv
2025-01-27 09:53:51,935 - INFO - DataFrame Shape: (95662, 16)


In [6]:
# Create an instance or object
features = FeatureEngineering(data)

In [7]:
# Aggregate Features
aggregate_features = features.create_aggregate_features()
print("Aggregated Features:")
aggregate_features.head(2)

Aggregated Features:


,CustomerId,total_transaction_amount,average_transaction_amount,transaction_count,transaction_amount_std
0,CustomerId_1,-10000.0,-10000.0,1,NaN
1,CustomerId_10,-10000.0,-10000.0,1,NaN


In [8]:
# Extract Date Features
data= features.extract_date_features()
print("\nDataFrame with Date Features:")
data.head(3)


DataFrame with Date Features:


,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult,transaction_hour,transaction_day,transaction_month,transaction_year
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15 02:18:49+00:00,2,0,2,15,11,2018
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15 02:19:08+00:00,2,0,2,15,11,2018
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15 02:44:21+00:00,2,0,2,15,11,2018


In [9]:
# Encode Categorical Variables
data = features.encode_categorical_variables(method='label')
print("\nDataFrame with Encoded Categorical Variables:")
data.head(3)


DataFrame with Encoded Categorical Variables:


,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult,transaction_hour,transaction_day,transaction_month,transaction_year
0,TransactionId_76871,46980,2490,3535,CustomerId_4406,0,256,5,1,0,2,1000.0,1000,2018-11-15 02:18:49+00:00,2,0,2,15,11,2018
1,TransactionId_73770,31755,3219,2366,CustomerId_4406,0,256,3,19,2,1,-20.0,20,2018-11-15 02:19:08+00:00,2,0,2,15,11,2018
2,TransactionId_26203,60272,2713,996,CustomerId_4683,0,256,5,0,0,2,500.0,500,2018-11-15 02:44:21+00:00,2,0,2,15,11,2018


In [10]:
# Scale Numerical Features
# If you have significant outliers, consider standardization or robust 
# scaling methods that are less affected by outliers also it is when dealing with numerical data that includes negative values.
data = features.scale_numerical_features(method='standardize')
print("\nDataFrame with Scaled Numerical Features:")
data.head(3)


DataFrame with Scaled Numerical Features:


,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult,transaction_hour,transaction_day,transaction_month,transaction_year
0,TransactionId_76871,46980,2490,3535,CustomerId_4406,0,256,5,1,0,2,-0.046371,-0.072291,2018-11-15 02:18:49+00:00,2,0,2,15,11,2018
1,TransactionId_73770,31755,3219,2366,CustomerId_4406,0,256,3,19,2,1,-0.054643,-0.080251,2018-11-15 02:19:08+00:00,2,0,2,15,11,2018
2,TransactionId_26203,60272,2713,996,CustomerId_4683,0,256,5,0,0,2,-0.050426,-0.076352,2018-11-15 02:44:21+00:00,2,0,2,15,11,2018


# Modelling

In [17]:
# Define target and features
target_transaction = 'FraudResult'
features_transaction = [
    'Amount', 'Value', 'transaction_hour', 'transaction_day', 'transaction_month',
    'transaction_year', 'CountryCode', 'ProviderId', 'ProductId', 'ProductCategory'
]

# Initialize model pipeline
transaction_model = CreditScoringModel(
    df=data,  
    target_col=target_transaction,
    feature_cols=features_transaction
)

# Split, train, tune, evaluate, and save
X_train, X_test, y_train, y_test = transaction_model.split_data()
transaction_model.train_models(X_train, y_train)
transaction_model.hyperparameter_tuning(X_train, y_train)
metrics_transaction = transaction_model.evaluate_models(X_test, y_test)
transaction_model.save_model('transaction_level_model.pkl')

print("Transaction-Level Model Metrics:")
for model, metric in metrics_transaction.items():
    print(f"{model}: {metric}")


Training LogisticRegression...
Training DecisionTree...
Training RandomForest...
Training GBM...
Tuning RandomForest...
Tuning GBM...
Best Model: RandomForest
Best model saved to transaction_level_model.pkl
Transaction-Level Model Metrics:
LogisticRegression: {'Accuracy': 0.9979093712434014, 'Precision': 0.4827586206896552, 'Recall': 0.358974358974359, 'F1 Score': 0.4117647058823529, 'ROC-AUC': 0.9948527259200769}
DecisionTree: {'Accuracy': 0.9993205456541054, 'Precision': 0.8611111111111112, 'Recall': 0.7948717948717948, 'F1 Score': 0.8266666666666667, 'ROC-AUC': 0.9101429365648491}
RandomForest: {'Accuracy': 0.9994250770919354, 'Precision': 0.9375, 'Recall': 0.7692307692307693, 'F1 Score': 0.8450704225352113, 'ROC-AUC': 0.9995266334168607}
GBM: {'Accuracy': 0.9994773428108503, 'Precision': 0.967741935483871, 'Recall': 0.7692307692307693, 'F1 Score': 0.8571428571428571, 'ROC-AUC': 0.9673316627857321}


# Predict on the given data

In [22]:
# Load the trained model 
MODEL_PATH = "../notebooks/transaction_level_model.pkl"

if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError(f"Trained model file not found at {MODEL_PATH}")
import joblib
model = joblib.load(MODEL_PATH)
input_data = data[features_transaction]

# Step 5: Make predictions
predictions = model.predict(input_data)

# Return predictions as a JSON response
print(f"predictions {predictions.tolist()}") 

predictions [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 